<h1 align="center">Comment faire une étude de zone de chalandise ?</h1>

Le tutoriel complet est dans le [listing d'outil](https://www.notion.so/Listing-d-outils-f7843c0adbd643e4824edff147b35ab3?pvs=4#c0b9ca6d04b84da4a31e09ac42bc68ee) sur Notion.

In [1]:
#@markdown **Initialisation**

#@markdown *Exécuter la cellule*

#@markdown Si une page apparaît, appuyer sur "Se connecter à Google Drive" et valider toutes les étapes.

%%capture

from google.colab import drive,widgets
import os
import sys

drive.mount('/content/gdrive')
%cd '/content/gdrive/Shareddrives/FoodBiome - Team/5. Ressources/Cartographies/Outil : Cartographie zone de chalandise/zone-de-chalandise/src/script'

sys.path.append("/content/gdrive/Shareddrives/FoodBiome - Team/5. Ressources/Cartographies/Outil : Cartographie zone de chalandise/venv_zone_de_chalandise/lib/python3.10/site-packages")


# remove files from previous runs
def remove_files_with_extensions(folder_path, extensions):
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            file_path = os.path.join(root, filename)
            if any(filename.lower().endswith(ext) for ext in extensions):
                try:
                    os.remove(file_path)
                    print(f"Removed: {file_path}")
                except Exception as e:
                    print(f"Error removing {file_path}: {e}")

# Specify the parent folder and file extensions to remove
parent_folder_path = '/content/gdrive/Shareddrives/FoodBiome - Team/5. Ressources/Cartographies/Outil : Cartographie zone de chalandise/zone-de-chalandise'
to_remove = ["kml", "geojson", "csv", "html",'txt']

# Call the function to remove files with specified extensions
remove_files_with_extensions(parent_folder_path, to_remove)


In [2]:

#@title ## Création de la carte


#@markdown ### 1. Choisir les entreprises cibles

#@markdown Copier les **codes NAF** des entreprises désirées

#@markdown *ex : 47.11F pour les Hypermarchés*

#@markdown Un document pour trouver ces codes est disponible [ici](https://docs.google.com/spreadsheets/d/1w-qYQrkUtMocTPsAU3vc3updnXwe_8Xc_9_v2-p04sE/edit#gid=1175193769).


code_naf_1 = '10.83Z'  #@param {type:"string"}
code_naf_2 = '47.11F'  #@param {type:"string"}
code_naf_3 = '47.11D'  #@param {type:"string"}
code_naf_4 = ''  #@param {type:"string"}
code_naf_5 = ''  #@param {type:"string"}
code_naf_6 = ''  #@param {type:"string"}
code_naf_7 = ''  #@param {type:"string"}
code_naf_8 = ''  #@param {type:"string"}
code_naf_9 = ''  #@param {type:"string"}
code_naf_10 = ''  #@param {type:"string"}



# Store the selected options in a list 'naf'
selected_naf = [code_naf_1, code_naf_2, code_naf_3, code_naf_4, code_naf_5, code_naf_6, code_naf_7, code_naf_8, code_naf_9, code_naf_10]

# remove empty
selected_naf = [code for code in selected_naf if code != '']

#@markdown **Une fois que c'est fait**, exécuter la cellule



In [3]:
#@markdown ### 2. Choisir le fichier de sortie désiré


#@markdown  - **Option 1 : Carte toute faite**
#@markdown  - prête à l'usage
#@markdown  - non modifiable
#@markdown  - voir [tuto](https://www.notion.so/Listing-d-outils-f7843c0adbd643e4824edff147b35ab3?pvs=4#d62aec487dc349eab439b563a1100489)



#@markdown  - **Option 2 : Carte MyMaps**
#@markdown  - carte modifiable à construire
#@markdown  - voir [tuto](https://www.notion.so/Listing-d-outils-f7843c0adbd643e4824edff147b35ab3?pvs=4#ba254c6deb9d41718d32fbfc3c9e96eb)

Option_1 = True #@param {type:"boolean"}
Option_2 = True #@param {type:"boolean"}

#@markdown **Une fois que c'est fait**, exécuter la cellule



In [4]:

#@markdown ### 3. Uploader le fichier d'aire téléchargé

#@markdown *Exécuter la cellule*


#@markdown Par exemple le fichier geojson de Géoportail


#@markdown *Bouton "Sélect. fichiers" en bas à gauche*



from google.colab import files
from main import _main,polygon_to_kml
import os
from IPython.display import display, HTML
from IPython.utils import io
import pandas as pd
import ipywidgets


naf = pd.read_parquet("../../data/Données nationales/NAF.parquet")

# libellés naf correspondants aux codes choisis
libelle = naf[naf.libelleNaf.isin(selected_naf)].libelleNaf.to_list()


# Function to be executed when the file is uploaded
def on_file_upload(uploaded_files):
    return uploaded_files


upload_button = files.upload()

with io.capture_output() as captured:
  display(upload_button)


uploaded = on_file_upload(upload_button)

for name, data in uploaded.items():
  with open('../../data/Données sites/'+name, 'wb') as f:
    f.write(data)


# Call your _main function with the necessary parameters
m, df,polygon,population = _main(kml_file_path='../../data/Données sites/'+name, naf=selected_naf,companies_file_path = "../../data/Données nationales/RNE_Sirene_localisé.parquet")

if Option_1 :
# Save the HTML map

  m.save('../../data/Données sites/'+name.split('.')[0] +'.html')

  files.download('../../data/Données sites/'+name.split('.')[0] +'.html')


if Option_2:

  df.columns = ["siret","qualite_xy","latitude","longitude","code NAF","nom commercial"," code commune Insee","adresse","diffusion commerciale",	"confiance dans la localisation",	"lien Pappers"]

  df.to_csv('../../data/Données sites/'+name.split('.')[0] +'.csv',index=False)

  files.download('../../data/Données sites/'+name.split('.')[0] +'.csv')

  polygon_to_kml(polygon,'../../data/Données sites/'+name.split('.')[0] +'.kml')

  files.download('../../data/Données sites/'+name.split('.')[0] +'.kml')

  with open('../../data/Données sites/'+name.split('.')[0] +'.txt','w') as file:
    file.write("Population du site : "+str(population)+" habitants")

  files.download('../../data/Données sites/'+name.split('.')[0] +'.txt')


#@markdown *NB: le processus peut prendre plusieurs minutes*



Saving CC de Saint-Flour.kml to CC de Saint-Flour.kml
Lecture du fichier de surface...
Filtre des communes dans l'aire d'étude...
Filtre des entreprises dans l'aire d'étude...
Création de la carte...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>